In [1]:
from google.colab import drive
import joblib

drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/auto_parts_models/xgboost_auto_parts_model.pkl'
preprocessor_path = '/content/drive/MyDrive/auto_parts_models/xgboost_preprocessor.pkl'

final_model = joblib.load(model_path)
preprocessor = joblib.load(preprocessor_path)

print("Model and preprocessor loaded successfully.")



Mounted at /content/drive
Model and preprocessor loaded successfully.


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import calendar
import joblib
from google.colab import drive
from IPython.display import display

# Mount Google Drive
drive.mount('/content/drive')

# Load model and preprocessor
model_path = '/content/drive/MyDrive/auto_parts_models/xgboost_auto_parts_model.pkl'
preprocessor_path = '/content/drive/MyDrive/auto_parts_models/xgboost_preprocessor.pkl'
final_model = joblib.load(model_path)
preprocessor = joblib.load(preprocessor_path)

# Generate synthetic historical data
products = [f"P{str(i).zfill(4)}" for i in range(1, 301)]
product_names = [f"Part_{i}" for i in range(1, 301)]
categories = ['Engine', 'Brake', 'Suspension']
brands = ['Denso', 'Bosch', 'Valeo']
car_models = ['Toyota', 'Nissan', 'Honda']

start_date = datetime.today() - relativedelta(years=3)
dates = pd.date_range(start=start_date, periods=36, freq='MS')

synthetic_data = []
for pid, pname in zip(products, product_names):
    category = np.random.choice(categories)
    brand = np.random.choice(brands)
    car_model = np.random.choice(car_models)
    unit_price = np.round(np.random.uniform(100, 1000), 2)
    for date in dates:
        units_sold = np.random.randint(1, 20)
        revenue = units_sold * unit_price
        synthetic_data.append([
            date, pid, pname, category, brand, car_model,
            unit_price, units_sold, revenue
        ])

df = pd.DataFrame(synthetic_data, columns=[
    'date', 'product_id', 'product_name', 'category', 'brand', 'car_model',
    'unit_price', 'units_sold', 'revenue'
])

# Feature engineering
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week.astype(int)
df['dayofweek'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter
df['product_category'] = df['product_name'] + '_' + df['category']
df['brand_model'] = df['brand'] + '_' + df['car_model']
df['category_model'] = df['category'] + '_' + df['car_model']
df = df.sort_values(by=['product_id', 'date'])
df['rolling_3m_units'] = df.groupby('product_id')['units_sold'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df['rolling_7m_units'] = df.groupby('product_id')['units_sold'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
df['rolling_3m_revenue'] = df.groupby('product_id')['revenue'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df['rolling_7m_revenue'] = df.groupby('product_id')['revenue'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
df.fillna(0, inplace=True)

# Prediction interface
def highlight(val):
    return 'background-color: #2e2e2e; color: #aaf; font-weight: bold; border: 1px solid #444;'

def predict_interface():
    all_ids = sorted(df['product_id'].unique())
    print("Available Product IDs:")
    for i in range(0, len(all_ids), 10):
        print("\t" + "\t".join(all_ids[i:i+10]))

    product_input = input("\nEnter Product IDs (comma-separated) or press Enter for all: ").strip()
    if product_input == "":
        target_ids = all_ids
    else:
        target_ids = [x.strip() for x in product_input.split(",") if x.strip() in all_ids]
        if not target_ids:
            print("❌ Invalid or missing Product IDs.")
            return

    try:
        months = int(input("Enter number of months to predict (1-12): ").strip())
        if not 1 <= months <= 12:
            raise ValueError
    except:
        print("❌ Invalid number of months.")
        return

    future_dfs = []
    today = datetime.today()
    for pid in target_ids:
        product_df = df[df['product_id'] == pid].copy()
        product_df = product_df.sort_values(by='date')
        recent = product_df.tail(9).copy()
        last_date = recent['date'].max()

        new_rows = []
        for i in range(months):
            date = (last_date + relativedelta(months=i+1)).replace(day=1)
            row = recent.iloc[-1].copy()
            row['date'] = date
            row['year'] = date.year
            row['month'] = date.month
            row['week'] = date.isocalendar()[1]
            row['dayofweek'] = date.weekday()
            row['quarter'] = (date.month - 1) // 3 + 1
            new_rows.append(row)

        future_df = pd.DataFrame(new_rows)
        future_df['product_category'] = future_df['product_name'] + '_' + future_df['category']
        future_df['brand_model'] = future_df['brand'] + '_' + future_df['car_model']
        future_df['category_model'] = future_df['category'] + '_' + future_df['car_model']
        future_df['rolling_3m_units'] = recent['rolling_3m_units'].mean()
        future_df['rolling_7m_units'] = recent['rolling_7m_units'].mean()
        future_df['rolling_3m_revenue'] = recent['rolling_3m_revenue'].mean()
        future_df['rolling_7m_revenue'] = recent['rolling_7m_revenue'].mean()

        features = [
            'product_name', 'category', 'brand', 'car_model', 'unit_price',
            'year', 'month', 'week', 'quarter', 'dayofweek',
            'product_category', 'category_model', 'brand_model',
            'rolling_3m_units', 'rolling_7m_units', 'rolling_3m_revenue', 'rolling_7m_revenue'
        ]

        X_future = preprocessor.transform(future_df[features])
        preds = final_model.predict(X_future)

        future_df['units_predicted'] = preds
        future_df['total_cost'] = preds * future_df['unit_price']
        future_df['month_name'] = future_df['month'].apply(lambda m: calendar.month_name[m]) + " " + future_df['year'].astype(str)

        out = future_df[['product_id', 'month_name', 'units_predicted', 'unit_price', 'total_cost']].copy()
        out[['units_predicted', 'unit_price', 'total_cost']] = out[['units_predicted', 'unit_price', 'total_cost']].round(0).astype(int)
        future_dfs.append(out)

    final_result = pd.concat(future_dfs).reset_index(drop=True)
    styled = final_result.style.set_table_styles(
        [{'selector': 'thead th', 'props': [('background-color', '#444'), ('color', '#fff')]}]
    ).map(highlight)

    print(f"\n📊 Prediction Summary for {months} Month(s):")
    display(styled)

predict_interface()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Available Product IDs:
	P0001	P0002	P0003	P0004	P0005	P0006	P0007	P0008	P0009	P0010
	P0011	P0012	P0013	P0014	P0015	P0016	P0017	P0018	P0019	P0020
	P0021	P0022	P0023	P0024	P0025	P0026	P0027	P0028	P0029	P0030
	P0031	P0032	P0033	P0034	P0035	P0036	P0037	P0038	P0039	P0040
	P0041	P0042	P0043	P0044	P0045	P0046	P0047	P0048	P0049	P0050
	P0051	P0052	P0053	P0054	P0055	P0056	P0057	P0058	P0059	P0060
	P0061	P0062	P0063	P0064	P0065	P0066	P0067	P0068	P0069	P0070
	P0071	P0072	P0073	P0074	P0075	P0076	P0077	P0078	P0079	P0080
	P0081	P0082	P0083	P0084	P0085	P0086	P0087	P0088	P0089	P0090
	P0091	P0092	P0093	P0094	P0095	P0096	P0097	P0098	P0099	P0100
	P0101	P0102	P0103	P0104	P0105	P0106	P0107	P0108	P0109	P0110
	P0111	P0112	P0113	P0114	P0115	P0116	P0117	P0118	P0119	P0120
	P0121	P0122	P0123	P0124	P0125	P0126	P0127	P0128	P0129	P0130
	P0131	P0132	P0133	P0134	P0135	P0136	P0137	P0138	P0139	

,product_id,month_name,units_predicted,unit_price,total_cost
0,P0162,August 2025,11,300,3192
1,P0162,September 2025,11,300,3216
2,P0162,October 2025,11,300,3230
3,P0162,November 2025,12,300,3457
4,P0162,December 2025,13,300,3758
5,P0162,January 2026,13,300,3842
6,P0162,February 2026,13,300,3831
